In [4]:

import pandas as pd
#Load shuttle dataset
df = pd.read_csv("C:/Users/61411/OneDrive - Deakin University/Desktop/Local_Katabatic_Repository/Katabatic/data/shuttle/shuttle.trn", delim_whitespace=True, header=None)
df.head()


C:\Users\61411\AppData\Local\Temp\ipykernel_34420\532753869.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("C:/Users/61411/OneDrive - Deakin University/Desktop/Local_Katabatic_Repository/Katabatic/data/shuttle/shuttle.trn", delim_whitespace=True, header=None)


,0,1,2,3,4,5,6,7,8,9
0,50,21,77,0,28,0,27,48,22,2
1,55,0,92,0,0,26,36,92,56,4
2,53,0,82,0,52,-5,29,30,2,1
3,37,0,76,0,28,18,40,48,8,1
4,37,0,79,0,34,-26,43,46,2,1


In [5]:

# Import required libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler


In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler

# 1. Data Preparation
class ShuttleDataset(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path, delim_whitespace=True, header=None)
        self.scaler = StandardScaler()
        
        # Last column is the label
        self.features = self.data.iloc[:, :-1].values
        self.labels = self.data.iloc[:, -1].values - 1
        
        self.features = self.scaler.fit_transform(self.features)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.features[idx]), torch.LongTensor([self.labels[idx]])

# Load the Shuttle dataset (.trn file)
dataset = ShuttleDataset("C:/Users/61411/OneDrive - Deakin University/Desktop/Local_Katabatic_Repository/Katabatic/data/shuttle/shuttle.trn")
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# 2. Define Network Components
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, z):
        return self.fc(z)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

# Initialize models
latent_dim = 100
generator = Generator(latent_dim, dataset.features.shape[1])
discriminator = Discriminator(dataset.features.shape[1])

# 3. Training Loop
def train_cramer_gan(generator, discriminator, dataloader, epochs=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    discriminator.to(device)
    
    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        for real_data, _ in dataloader:
            real_data = real_data.to(device)
            batch_size = real_data.size(0)
            
            # Train Discriminator
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_data = generator(z).detach()
            
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            loss_real = criterion(discriminator(real_data), real_labels)
            loss_fake = criterion(discriminator(fake_data), fake_labels)
            loss_d = loss_real + loss_fake
            loss_d.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_data = generator(z)
            loss_g = criterion(discriminator(fake_data), real_labels)
            loss_g.backward()
            optimizer_g.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss D: {loss_d.item()}, Loss G: {loss_g.item()}')

# Start Training
train_cramer_gan(generator, discriminator, dataloader)



C:\Users\61411\AppData\Local\Temp\ipykernel_34420\2235807836.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  self.data = pd.read_csv(file_path, delim_whitespace=True, header=None)


Epoch [1/100], Loss D: 0.7294900417327881, Loss G: 1.514638900756836
Epoch [2/100], Loss D: 0.3036311864852905, Loss G: 2.7119061946868896
Epoch [3/100], Loss D: 0.9956610202789307, Loss G: 1.7124689817428589
Epoch [4/100], Loss D: 0.8497650623321533, Loss G: 1.3429328203201294
Epoch [5/100], Loss D: 1.0902647972106934, Loss G: 0.9602717757225037
Epoch [6/100], Loss D: 0.9476759433746338, Loss G: 1.0110349655151367
Epoch [7/100], Loss D: 0.9747903943061829, Loss G: 0.9251346588134766
Epoch [8/100], Loss D: 1.034498691558838, Loss G: 0.9583221673965454
Epoch [9/100], Loss D: 1.1529650688171387, Loss G: 0.9314464330673218
Epoch [10/100], Loss D: 1.0188404321670532, Loss G: 0.9554740786552429
Epoch [11/100], Loss D: 1.0794105529785156, Loss G: 1.0219515562057495
Epoch [12/100], Loss D: 1.0655839443206787, Loss G: 0.9548749327659607
Epoch [13/100], Loss D: 1.0265991687774658, Loss G: 1.015663981437683
Epoch [14/100], Loss D: 1.0152918100357056, Loss G: 1.067575216293335
Epoch [15/100], Los

In [7]:
# Generate synthetic data
generator.eval()
num_samples = len(dataset)
z = torch.randn(num_samples, latent_dim).to('cpu')  # Match real data count
synthetic_data = generator(z).detach().numpy()
real_labels = dataset.labels  # from the Dataset class
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Train and evaluate
models = {
    "LR": LogisticRegression(),
    "MLP": MLPClassifier(),
    "RF": RandomForestClassifier(),
    "XGBT": XGBClassifier()
}

for name, model in models.items():
    model.fit(synthetic_data, real_labels)
    preds = model.predict(dataset.features)  # Evaluate on real data
    acc = accuracy_score(real_labels, preds)
    print(f"{name}: {acc:.4f}")


LR: 0.7837
MLP: 0.7841
RF: 0.7451
XGBT: 0.7302


In [8]:

synthetic_df = pd.DataFrame(synthetic_data)

# View the first few synthetic samples
synthetic_df.head()


,0,1,2,3,4,5,6,7,8
0,-0.209404,-0.063460,-0.344881,0.065880,0.494913,0.010283,-0.042471,-0.638542,-0.520761
1,-0.915200,-0.021878,-0.908336,0.012734,-0.471409,0.030079,0.267926,0.124677,-0.067652
2,0.194794,-0.048422,0.151622,0.018783,0.663420,-0.061387,-0.087623,-0.561768,-0.466256
3,-0.505625,-0.043630,-0.375797,0.013232,0.345206,0.017632,0.201714,-0.436891,-0.527415
4,-0.483685,-0.053000,0.184205,0.014867,0.370311,-0.018831,0.589455,-0.219688,-0.505142


In [9]:
print(synthetic_data.shape)


(43500, 9)
